#### Generating the raw dataframes

In [ ]:
import pandas as pd
import yfinance as yf

# Obtain dataframe with monthly closes of the standards

VWCE_data = yf.Ticker("VWCE.DE").history(start="2022-11-01")
VWCE_data_montly = VWCE_data.resample('ME').last()
VWCE_data_montly = VWCE_data_montly.reset_index()

VUSA_data = yf.Ticker("VUSA.AS").history(start="2022-11-01")
VUSA_data_montly = VUSA_data.resample('ME').last()
VUSA_data_montly = VUSA_data_montly.reset_index()

df_vwce = VWCE_data_montly [["Date", "Close"]].rename(columns = {"Date": "date", "Close": "vwce_close"})
df_vusa = VUSA_data_montly [["Date", "Close"]].rename(columns = {"Date": "date", "Close": "vusa_close"})

# Calculate monthly variations of standards

df_vwce.loc [0, "vwce_value_of_100eur"] = 100.0

for i in range (1, len(df_vwce)):
    df_vwce.loc [i, "vwce_absol_variation"] = df_vwce.loc [i, "vwce_close"] - df_vwce.loc [i-1, "vwce_close"]
    df_vwce.loc [i, "vwce_rel_variation"] = df_vwce.loc [i, "vwce_absol_variation"] / df_vwce.loc [i-1, "vwce_close"]
    df_vwce.loc [i, "vwce_value_of_100eur"] = df_vwce.loc [i-1, "vwce_value_of_100eur"] * (1 + df_vwce.loc [i, "vwce_rel_variation"])

df_vusa.loc [0, "vusa_value_of_100eur"] = 100.0

for i in range (1, len(df_vusa)):
    df_vusa.loc [i, "vusa_absol_variation"] = df_vusa.loc [i, "vusa_close"] - df_vusa.loc [i-1, "vusa_close"]
    df_vusa.loc [i, "vusa_rel_variation"] = df_vusa.loc [i, "vusa_absol_variation"] / df_vusa.loc [i-1, "vusa_close"]
    df_vusa.loc [i, "vusa_value_of_100eur"] = df_vusa.loc [i-1, "vusa_value_of_100eur"] * (1 + df_vusa.loc [i, "vusa_rel_variation"])

# Import demo dataframe

df_wallet = pd.read_csv("demo_raw_dataframe.csv", delimiter=";")

# Calculate monthly variation

df_wallet.loc [0, "wallet_value_of_100eur"] = 100.0
df_wallet = df_wallet.fillna(0)

for i in range (1, len(df_wallet)):
    df_wallet.loc [i, "wallet_absol_variation"] = df_wallet.loc [i, "wallet_close"] - df_wallet.loc [i-1, "wallet_close"] - df_wallet.loc [i, "wallet_net_invested"]
    df_wallet.loc [i, "wallet_rel_variation"] = df_wallet.loc [i, "wallet_absol_variation"] / df_wallet.loc [i-1, "wallet_close"]
    df_wallet.loc [i, "wallet_value_of_100eur"] = df_wallet.loc [i-1, "wallet_value_of_100eur"] * (1 + df_wallet.loc [i, "wallet_rel_variation"])

# Reorder columns

df_vwce = df_vwce [["date", "vwce_close", "vwce_absol_variation", "vwce_rel_variation", "vwce_value_of_100eur"]]
df_vusa = df_vusa [["date", "vusa_close", "vusa_absol_variation", "vusa_rel_variation", "vusa_value_of_100eur"]]
df_wallet = df_wallet [["date", "wallet_net_invested", "wallet_close", "wallet_absol_variation", "wallet_rel_variation", "wallet_value_of_100eur"]]

# Exporting dataframes

df_vwce.to_csv("exported_dataframes/df_vwce.csv", encoding='utf-8-sig')
df_vusa.to_csv("exported_dataframes/df_vusa.csv", encoding='utf-8-sig')
df_wallet.to_csv("exported_dataframes/df_wallet.csv", encoding='utf-8-sig')


#### Generating the monthly evolution dataset

In [66]:
df_evolution_month = df_vusa.copy()
df_evolution_month [["vwce_close", "vwce_absol_variation", "vwce_rel_variation", "vwce_value_of_100eur"]] = df_vwce [["vwce_close", "vwce_absol_variation", "vwce_rel_variation", "vwce_value_of_100eur"]]
df_evolution_month [["wallet_net_invested", "wallet_close", "wallet_absol_variation", "wallet_rel_variation", "wallet_value_of_100eur"]] = df_wallet [["wallet_net_invested", "wallet_close", "wallet_absol_variation", "wallet_rel_variation", "wallet_value_of_100eur"]]

df_evolution_month.to_csv("exported_dataframes/df_evolution_month.csv", encoding='utf-8-sig')

#df_evolution_month

#### Generating the annual dataset

In [92]:
dataframe_yearly = pd.read_csv("exported_dataframes/df_evolution_month.csv", index_col=0)
dataframe_yearly["date"] = pd.to_datetime (df_evolution_month["date"], utc=True)

list_all_indexes = [i for i in range (0,len(dataframe_yearly))]
list_index_jans = dataframe_yearly.index[dataframe_yearly["date"].dt.month == 1].tolist()
list_except_jans = [i for i in list_all_indexes if i not in list_index_jans]

for i in range (1, len(dataframe_yearly)):
    dataframe_yearly.loc [i, "vwce_open"] = dataframe_yearly.loc [i-1, "vwce_value_of_100eur"]
    dataframe_yearly.loc [i, "vusa_open"] = dataframe_yearly.loc [i-1, "vusa_value_of_100eur"]
    dataframe_yearly.loc [i, "wallet_open"] = dataframe_yearly.loc [i-1, "wallet_value_of_100eur"]

dataframe_yearly = dataframe_yearly [["date", "vwce_open", "vwce_value_of_100eur", "vwce_rel_variation", "vusa_open", "vusa_value_of_100eur", "vusa_rel_variation", "wallet_open", "wallet_value_of_100eur", "wallet_rel_variation"]]

#Define opens of Jan as 100, and then the closing values considering that new beginning
dataframe_yearly.loc [list_index_jans,["vwce_open", "vusa_open", "wallet_open"]] = 100
dataframe_yearly.loc [list_index_jans,"vwce_value_of_100eur"] = dataframe_yearly.loc[list_index_jans,"vwce_open"] * (1 + (dataframe_yearly.loc[list_index_jans,"vwce_rel_variation"]) )
dataframe_yearly.loc [list_index_jans,"vusa_value_of_100eur"] = dataframe_yearly.loc[list_index_jans,"vusa_open"] * (1 + (dataframe_yearly.loc[list_index_jans,"vusa_rel_variation"]) )
dataframe_yearly.loc [list_index_jans,"wallet_value_of_100eur"] = dataframe_yearly.loc[list_index_jans,"wallet_open"] * (1 + (dataframe_yearly.loc[list_index_jans,"wallet_rel_variation"]) )

# Apply the same filter in the remaining rows
for i in list_except_jans[1:]:
    dataframe_yearly.loc [i,"vwce_open"] = dataframe_yearly.loc [i-1,"vwce_value_of_100eur"]
    dataframe_yearly.loc [i,"vwce_value_of_100eur"] = dataframe_yearly.loc[i,"vwce_open"] * (1 + (dataframe_yearly.loc[i,"vwce_rel_variation"]) )

    dataframe_yearly.loc [i,"vusa_open"] = dataframe_yearly.loc [i-1,"vusa_value_of_100eur"]
    dataframe_yearly.loc [i,"vusa_value_of_100eur"] = dataframe_yearly.loc[i,"vusa_open"] * (1 + (dataframe_yearly.loc[i,"vusa_rel_variation"]) )

    dataframe_yearly.loc [i,"wallet_open"] = dataframe_yearly.loc [i-1,"wallet_value_of_100eur"]
    dataframe_yearly.loc [i,"wallet_value_of_100eur"] = dataframe_yearly.loc[i,"wallet_open"] * (1 + (dataframe_yearly.loc[i,"wallet_rel_variation"]) )

# Filter the dataframe to include only the last month of the year
dataframe_yearly = dataframe_yearly.reset_index()
dataframe_yearly = dataframe_yearly.set_index("date")
dataframe_yearly = dataframe_yearly.resample('YE').last()

dataframe_yearly.to_csv("exported_dataframes/dataframe_yearly.csv", encoding='utf-8-sig')

dataframe_yearly

,index,vwce_open,vwce_value_of_100eur,vwce_rel_variation,vusa_open,vusa_value_of_100eur,vusa_rel_variation,wallet_open,wallet_value_of_100eur,wallet_rel_variation
date,,,,,,,,,,
2022-12-31 00:00:00+00:00,1,100.000000,94.513401,-0.054866,100.000000,93.301347,-0.066987,100.000000,95.607696,-0.043923
2023-12-31 00:00:00+00:00,13,113.828494,118.176803,0.038201,118.183725,122.120924,0.033314,107.048735,113.940804,0.064383
2024-12-31 00:00:00+00:00,25,125.663051,124.411654,-0.009958,134.212741,133.862402,-0.002610,104.851787,103.803526,-0.009998
2025-12-31 00:00:00+00:00,36,109.247855,108.332087,-0.008382,104.920918,104.060439,-0.008201,111.867357,111.485943,-0.003410
